In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.torch.attacks.projected_gradient_descent import (
    projected_gradient_descent,
)
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms

In [3]:
from quantus.functions.perturb_func import baseline_replacement_by_indices

In [4]:
from art.attacks.evasion import CarliniLInfMethod
import torch.optim as optim
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

# Attack functions

In [7]:
def make_attack(x_batch, y_batch, eps, normal_model): 
    
    alpha = eps/10
    steps = int(alpha*eps)
    images_pgd = projected_gradient_descent(normal_model, x_batch, eps, alpha, steps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [8]:
def make_fgsm_attack(x_batch, y_batch, eps, normal_model): 
    
    images_pgd = fast_gradient_method(normal_model, x_batch, eps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    index = (y_pred_pgd != y_batch)
    pgd_images = images_pgd[index]
    y_pred_pgd = y_pred_pgd[index]
    return pgd_images, y_pred_pgd

In [9]:
def make_cw_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

In [10]:
def make_bim_attack(attack, x_batch, y_batch, normal_model):
    #convert tensor to numpy
    #x_batch=x_batch.to('cuda')
    #_, y_pred_pgd = normal_model(x_batch).max(1)
    x_batch = x_batch.cpu().numpy()
    x_test_adv = attack.generate(x=x_batch)
    #convert the nd array back to tensor
    x_test = torch.from_numpy(x_test_adv).to('cuda')
    _, y_test = normal_model(x_test).max(1)
    index = (y_test != y_batch)
    adv_images = x_test[index]
    y_pred_adv = y_test[index]
    return adv_images, y_pred_adv

# Approach 2: Functions for sensitivity approachrobustness

In [12]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [14]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)

# Approach 1: Input squeeze 

In [7]:
#referenced from Quantus official library
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

def infer_attribution_axes(a_batch: np.ndarray, x_batch: np.ndarray)-> Sequence[int]:
    """
    Reference: quantus library
    Infers the axes in x_batch that are covered by a_batch.
    Parameters
    ----------
    x_batch: np.ndarray
        A np.ndarray which contains the input data that are explained.
    a_batch: np.ndarray
        An array which contains pre-computed attributions i.e., explanations.
    Returns
    -------
    np.ndarray
        The axes inferred.
    """
    # TODO: Adapt for batched processing.

    if a_batch.shape[0] != x_batch.shape[0]:
        raise ValueError(
            f"a_batch and x_batch must have same number of batches ({a_batch.shape[0]} != {x_batch.shape[0]})"
        )

    if a_batch.ndim > x_batch.ndim:
        raise ValueError(
            "Attributions need to have <= dimensions than inputs, but {} > {}".format(
                a_batch.ndim, x_batch.ndim
            )
        )

    # TODO: We currently assume here that the batch axis is not carried into the perturbation functions.
    a_shape = [s for s in np.shape(a_batch)[1:] if s != 1]
    x_shape = [s for s in np.shape(x_batch)[1:]]

    if a_shape == x_shape:
        return np.arange(0, len(x_shape))

    # One attribution value per sample
    if len(a_shape) == 0:
        return np.array([])

    x_subshapes = [
        [x_shape[i] for i in range(start, start + len(a_shape))]
        for start in range(0, len(x_shape) - len(a_shape) + 1)
    ]
    if x_subshapes.count(a_shape) < 1:

        # Check that attribution dimensions are (consecutive) subdimensions of inputs
        raise ValueError(
            "Attribution dimensions are not (consecutive) subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}".format(
                x_batch.shape, a_batch.shape
            )
        )
    elif x_subshapes.count(a_shape) > 1:

        # Check that attribution dimensions are (unique) subdimensions of inputs.
        # Consider potentially expanded dims in attributions.

        if a_batch.ndim == x_batch.ndim and len(a_shape) < a_batch.ndim:
            a_subshapes = [
                [np.shape(a_batch)[1:][i] for i in range(start, start + len(a_shape))]
                for start in range(0, len(np.shape(a_batch)[1:]) - len(a_shape) + 1)
            ]
            if a_subshapes.count(a_shape) == 1:

                # Inferring channel shape.
                for dim in range(len(np.shape(a_batch)[1:]) + 1):
                    if a_shape == np.shape(a_batch)[1:][dim:]:
                        return np.arange(dim, len(np.shape(a_batch)[1:]))
                    if a_shape == np.shape(a_batch)[1:][:dim]:
                        return np.arange(0, dim)

            raise ValueError(
                "Attribution axes could not be inferred for inputs of "
                "shape {} and attributions of shape {}".format(
                    x_batch.shape, a_batch.shape
                )
            )

        raise ValueError(
            "Attribution dimensions are not unique subdimensions of inputs:  "
            "inputs were of shape {} and attributions of shape {}."
            "Please expand attribution dimensions for a unique solution".format(
                x_batch.shape, a_batch.shape
            )
        )
    else:
        # Infer attribution axes.
        for dim in range(len(x_shape) + 1):
            if a_shape == x_shape[dim:]:
                return np.arange(dim, len(x_shape))
            if a_shape == x_shape[:dim]:
                return np.arange(0, dim)

    raise ValueError(
        "Attribution axes could not be inferred for inputs of "
        "shape {} and attributions of shape {}".format(x_batch.shape, a_batch.shape)
    )


In [16]:
def modify_x_based_on_attribution(x_batch, a_batch_benign, d_type, top_k = 100):
    #get attribution axes
    a_axes = infer_attribution_axes(a_batch_benign, x_batch)
    perturbed_arr = []

    #modify each sample one-by-one
    for x,a in zip(x_batch, a_batch_benign):
        #flatten the attribution 
        a = a.flatten()
        #get indices of sorted attributions (ascending)
        a_ind = np.argsort(a)
        #get indices of top_k 
        a_ix = a_ind[-top_k:]
        if d_type=="adv":
            x = x.cpu()
            x_perturbed = baseline_replacement_by_indices(arr = x.detach().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.detach().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
        else:
            x_perturbed = baseline_replacement_by_indices(arr = x.cpu().numpy(), indices = a_ix, indexed_axes = a_axes, perturb_baseline = "black")
            if (x.cpu().numpy().flatten() != x_perturbed.flatten()).any():
                perturbed_arr.append(torch.from_numpy(x_perturbed))
    
    new_batch = torch.stack(perturbed_arr)
    return new_batch.to('cuda')

# Approach 3: Statistics

In [17]:
def compute_mean_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        avg = np.mean(a)
        deviation = a - avg 
        absolute_deviation = np.abs(deviation)
        result = np.mean(absolute_deviation)
        scores.append(result)
    return scores    

In [18]:
def compute_median_abs_dev(attr): 
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        med = np.median(a)
        deviation = a - med 
        abs_deviation = np.abs(deviation)
        result = np.median(abs_deviation)
        scores.append(result)
    return scores 

In [19]:
def compute_iqr(attr):
    #inter-quartile range
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = score_75 - score_25
        scores.append(score_qt)
    return scores
    

In [20]:
#relative measure of dispersion
def compute_coef_var(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        m = np.mean(a)
        st = np.std(attr[i])
        sc = m/st
        scores.append(sc)
    return scores

In [21]:
#relative measure of dispersion

## Coefficient of quartile dev

def compute_coef_iqr(attr):
    scores = []
    for i in range(len(attr)):
        a = attr[i].flatten()
        score_75 = np.percentile(a, 75)
        score_25 = np.percentile(a, 25)
        score_qt = (score_75 - score_25)/(score_75 + score_25)
        scores.append(score_qt)
    return scores
    

# imagenet dataset

In [8]:
def compute_benign(train_loader, normal_model): 
    #compute metrics for for benign imagenet for threshold computation 
    
    print("Computing metrics for benign")
    #statistics benign
    medianAbs_ben = []
    meanAbs_ben = []
    iqr_ben = []
    coef_var_ben=[]
    coef_iqr_ben = []

    #input squeeze benign
    logit_ben1 = []
    logit_ben2 = []
    logit_ben3 = []

    #attribution robustness
    benign_attribution_gaussian1 = []
    benign_attribution_gaussian2 = []
    benign_attribution_gaussian3 = []
    benign_attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    

    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: statistics
        x_logits = normal_model(x_batch)
        a_batch_benign = (Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        meanAbs_ben += compute_mean_abs_dev(a_batch_benign)
        medianAbs_ben += compute_median_abs_dev(a_batch_benign)
        iqr_ben += compute_iqr(a_batch_benign)
        coef_var_ben += compute_coef_var(a_batch_benign)
        coef_iqr_ben += compute_coef_iqr(a_batch_benign)
        
        #approach: input squeeze
        a_batch_benign = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=x_batch, target=y_batch).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 5000, d_type="not-adv")
        x_modified2 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 10000, d_type="not-adv")
        x_modified3 = modify_x_based_on_attribution(x_batch, a_batch_benign, top_k = 30000, d_type="not-adv")
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit_ben1.extend(diff1.detach().cpu().numpy())
        logit_ben2.extend(diff2.detach().cpu().numpy())
        logit_ben3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch_benign = quantus.explain(
            model=normal_model, inputs=x_batch, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(x_batch, y_batch, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(x_batch, y_batch, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(x_batch, y_batch, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(x_batch, y_batch)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_batch, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch_benign, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch_benign, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch_benign, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            benign_attribution_uniform.append(c)
        
        #save the results in csv
        if step > 200: #500
            break
    
    df = pd.DataFrame([
            medianAbs_ben,
            meanAbs_ben, 
            iqr_ben, 
            coef_var_ben, 
            coef_iqr_ben, 
            logit_ben1,
            logit_ben2,
            logit_ben3,
            benign_attribution_gaussian1,
            benign_attribution_gaussian2,
            benign_attribution_gaussian3,
            benign_attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform], index = [
            "Benign Median Absolute Dev", 
            "Benign Mean Absolute Dev", 
            "Benign IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze top 100",
            "Input squeeze top 150",
            "Input squeeze top 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        ])
    
    df.to_csv("Benign_IMAGENET_Metrics.csv")

In [28]:
def compute_pgd(train_loader, normal_model, eps): 
    
    print("Computing metrics for {} for PGD".format(eps))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    

    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 5000, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 10000, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 30000, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 1000:
            break
    
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform], index = [
            " Median Absolute Dev", 
            " Mean Absolute Dev", 
            " IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        ])
    
    path = "eps_"+str(eps)+"_IMAGENET_PGD.csv"
    df.to_csv(path)

In [29]:
def compute_fgsm(train_loader, normal_model, eps): 
    
    print("Computing metrics for {} for FGSM".format(eps))
    
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    

    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_fgsm_attack(x_batch, y_batch, eps, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 5000, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 10000, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 30000, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 1000:
            break
    
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform], index = [
            " Median Absolute Dev", 
            " Mean Absolute Dev", 
            " IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        ])
            
    path = "eps_"+str(eps)+"_IMAGENET_FGSM.csv"
    df.to_csv(path)
            

In [9]:
def compute_bim(train_loader, normal_model, eps): 
    
    print("Computing metrics for {} for BIM".format(eps))
    
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(3, 224, 224),
        nb_classes=1000,
)
    attack = BasicIterativeMethod(classifier, eps = eps, max_iter=int(eps*256*1.25))
    
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_bim_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 5000, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 10000, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 30000, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 1000:
            break
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform], index = [
            " Median Absolute Dev", 
            " Mean Absolute Dev", 
            " IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        ])
            
    path = "eps_"+str(eps)+"_IMAGENET_BIM.csv"
    df.to_csv(path)
            

In [10]:
def compute_cw(train_loader, normal_model, conf): 
    
    print("Computing metrics for {} for CW".format(conf))
    medianAbs = []
    meanAbs = []
    iqr= []
    coef_var=[]
    coef_iqr = []

    logit1 = []
    logit2 = []
    logit3 = []
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    
    #set attack parameters for BIM
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(normal_model.parameters(), lr=0.01)
    classifier = PyTorchClassifier(
    model=normal_model, clip_values=(0, 1), loss=criterion, optimizer=optimizer, input_shape=(3, 224, 224),
        nb_classes=1000,
)
    attack = CarliniLInfMethod(classifier, confidence = conf)
    
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        gc.collect()
        torch.cuda.empty_cache()
        
        #create adv samples    
        images_adv,y_pred_adv = make_cw_attack(attack, x_batch, y_batch, normal_model)
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        if len(images_adv) == 0:
            continue
    
        #approach: statistics
        x_logits = normal_model(images_adv)
        a_batch = (Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        meanAbs += compute_mean_abs_dev(a_batch)
        medianAbs += compute_median_abs_dev(a_batch)
        iqr += compute_iqr(a_batch)
        coef_var += compute_coef_var(a_batch)
        coef_iqr += compute_coef_iqr(a_batch)
        
        #approach: input squeeze
        a_batch = quantus.normalise_func.normalise_by_negative(Saliency(normal_model).attribute(inputs=images_adv, target=y_pred_adv).sum(axis=1).cpu().numpy())
        x_modified1 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 5000, d_type="adv")
        x_modified2 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 10000, d_type="adv")
        x_modified3 = modify_x_based_on_attribution(images_adv, a_batch, top_k = 30000, d_type="adv")
        
        x_logits1 = normal_model(x_modified1)
        x_logits2 = normal_model(x_modified2)
        x_logits3 = normal_model(x_modified3)
        diff1 = torch.norm(x_logits-x_logits1,p=1, dim=1)
        diff2 = torch.norm(x_logits-x_logits2,p=1, dim=1)
        diff3 = torch.norm(x_logits-x_logits3,p=1, dim=1)
        logit1.extend(diff1.detach().cpu().numpy())
        logit2.extend(diff2.detach().cpu().numpy())
        logit3.extend(diff3.detach().cpu().numpy())
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1)
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
    
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        #save the results in csv
        if len(iqr) > 1000:
            break
    df = pd.DataFrame([
            medianAbs,
            meanAbs, 
            iqr, 
            coef_var, 
            coef_iqr, 
            logit1,
            logit2,
            logit3,
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
            logit_uniform], index = [
            " Median Absolute Dev", 
            " Mean Absolute Dev", 
            " IQR", 
            "Coefficient of Variance",
            "Coef of IQR",
            "Input squeeze 100",
            "Input squeeze 150",
            "Input squeeze 200",
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Unifrom attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
            "Uniform logit robusntess",
        ])
            
    path = "confidence_"+str(conf)+"_IMAGENET_CW.csv"
    df.to_csv(path)
            

In [32]:
def load_imagenet_model():
    model=torchvision.models.mobilenet_v3_small(weights=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [11]:
def main_imagenet(images): 
    
    
    normal_model = load_imagenet_model()
    normal_model.to(device)
    normal_model.eval()
    
    #get dataset
    train = torchvision.datasets.ImageFolder(images, transform=valid_transform)
    train_loader = DataLoader(train, shuffle=True, batch_size = 5)

    #compute all attack for each hyperparameter one by one 
    epsilons = [8/255, 16/255, 32/255, 64/255]
    confidences = [0,50]
    compute_benign(train_loader, normal_model) 
    for eps in epsilons:
        compute_fgsm(train_loader, normal_model, eps)
        compute_pgd(train_loader, normal_model, eps)
        compute_bim(train_loader, normal_model, eps)
    
    for conf in confidences: 
        compute_cw(train_loader, normal_model, conf)


In [34]:
def main(): 
    print("Analysis of imagenet")
    images = '/home/db1702/Downloads/imagenet-mini/val/'
    main_imagenet(images)
   

In [35]:
main()

Analysis of imagenet
Computing metrics for benign
Computing metrics for 0.03137254901960784 for FGSM
Computing metrics for 0.03137254901960784 for PGD
Computing metrics for 0.03137254901960784 for BIM


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

C&W L_inf:   0%|          | 0/5 [00:00<?, ?it/s]

C&W L_inf:   0%|          | 0/5 [00:00<?, ?it/s]

C&W L_inf:   0%|          | 0/5 [00:00<?, ?it/s]

C&W L_inf:   0%|          | 0/5 [00:00<?, ?it/s]

C&W L_inf:   0%|          | 0/5 [00:00<?, ?it/s]